In [2]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, glob
import itertools as it
import networkx as nx

In [4]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

listeTirages=glob.glob("/Users/gilles/Box Sync/2015-Data/*-Tirage.pkl")
listeTirages200=tiragesNom("200Mo")
listeTirages20=tiragesNom("20Mo")
listeTirages1=tiragesNom("1Mo")
listeTirages50k=tiragesNom("50Ko")
listeTirages100k=tiragesNom("100Ko")


In [5]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [6]:
nomLexique=listeTirages100k[0]
lexique=lireLexique(nomLexique)
taille=lexique[lexique["tir1"]>0]["tir1"].count()
paradigmes=pd.pivot_table(lexique[lexique["tir1"]>0], values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()


In [7]:
def compPaire(c1,c2):
    syncretisms=[]
    c1Val=paradigmes[c1].notnull()
    c2Val=paradigmes[c2].notnull()
    c1Sur=paradigmes[c1].str.contains(",")
    c2Sur=paradigmes[c2].str.contains(",")
    l1=len(paradigmes[paradigmes[c1].notnull()])
    l2=len(paradigmes[paradigmes[c2].notnull()])
    paire=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
    lenDiff=len(paire[~paire[c1].str.contains(",") & ~paire[c2].str.contains(",")])
    if lenDiff>0:
        if debug:
            print u"%s ≠ %s"%(c1,c2)
            print "différence",lenDiff
            if lenDiff<12:
                print paire
    else:
        surAbondant=paire[paire[c1].str.contains(",") | paire[c2].str.contains(",")]
#            print "--------------------------------"
        if len(surAbondant)==0:
#                print u"%s = %s"%(c1,c2) 
            syncretisms.append(u"%s = %s"%(c1,c2))
        else:
            compatible=True
            for index,row in surAbondant.iterrows():
                if "," in row[c1]:
                    if "," in row[c2]:
                        if row[c1]!=row[c2]:
                            compatible=False
                    else:
                        if not row[c2] in row[c1].split(","):
                            compatible=False
                else:
                    if not row[c1] in row[c2].split(","):
                        compatible=False
            if compatible:
#                    print u"%s = %s"%(c1,c2)
                syncretisms.append(u"%s = %s"%(c1,c2))
            else:
                print u"%s ≠ %s"%(c1,c2)
                print surAbondant

In [22]:
len(paradigmes[paradigmes["pi2S"].notnull() & paradigmes["pi3S"].notnull() & (paradigmes["pi2S"]!=paradigmes["pi3S"])][["pi2S","pi3S"]])


0